### Install the _MLMD_ library

In [ ]:
# To use the latest publish `ml-metadata` library, you can run:
!pip install ml-metadata
# Then restart the Notebook kernel.

In [ ]:
from datetime import datetime
from ml_metadata import metadata_store
from ml_metadata.proto import metadata_store_pb2

In [ ]:
METADATA_STORE_HOST = "metadata-grpc-service.kubeflow" # default DNS of Kubeflow Metadata gRPC service.
METADATA_STORE_PORT = 8080

### Create MetadataStore connected to gRPC service and use it in Python

In [ ]:
mlmd_connection_config = metadata_store_pb2.MetadataStoreClientConfig(
        host=METADATA_STORE_HOST,
        port=METADATA_STORE_PORT,
    )
store = metadata_store.MetadataStore(mlmd_connection_config)

### Register artifact types

In [ ]:
# Create ArtifactTypes, e.g., Data and Model
data_type = metadata_store_pb2.ArtifactType()
data_type.name = "DataSet"
data_type.properties["day"] = metadata_store_pb2.INT
data_type.properties["split"] = metadata_store_pb2.STRING
data_type_id = store.put_artifact_type(data_type)
print(data_type_id)

model_type = metadata_store_pb2.ArtifactType()
model_type.name = "SavedModel"
model_type.properties["version"] = metadata_store_pb2.INT
model_type.properties["name"] = metadata_store_pb2.STRING
model_type_id = store.put_artifact_type(model_type)
print(model_type_id)

### Register execution types for all steps in the ML workflow

In [ ]:
# Create an ExecutionType, e.g., Trainer
trainer_type = metadata_store_pb2.ExecutionType()
trainer_type.name = "Trainer"
trainer_type.properties["state"] = metadata_store_pb2.STRING
trainer_type.properties["name"] = metadata_store_pb2.STRING
trainer_type_id = store.put_execution_type(trainer_type)
print(trainer_type_id)

### Create an artifact of DataSet ArtifactType

In [ ]:
# Create an input artifact of type DataSet
data_artifact = metadata_store_pb2.Artifact()
data_artifact.uri = 'path/to/data'
data_artifact.properties["day"].int_value = 1
data_artifact.properties["split"].string_value = 'train'
data_artifact.type_id = data_type_id
data_artifact_id = store.put_artifacts([data_artifact])[0]
print(data_artifact_id)

### Create an execution of the Trainer run

In [ ]:
# Register the Execution of a Trainer run
trainer_run = metadata_store_pb2.Execution()
trainer_run.type_id = trainer_type_id
trainer_run.properties["name"].string_value = "My Execution"
trainer_run.properties["state"].string_value = "RUNNING"
run_id = store.put_executions([trainer_run])[0]
print(run_id)

### Define the input event and read data

In [ ]:
# Define the input event
input_event = metadata_store_pb2.Event()
input_event.artifact_id = data_artifact_id
input_event.execution_id = run_id
input_event.type = metadata_store_pb2.Event.DECLARED_INPUT
# Record the input event in the metadata store
store.put_events([input_event])

### Declare the output artifact

In [ ]:
# Declare the output artifact of type SavedModel
model_artifact = metadata_store_pb2.Artifact()
model_artifact.uri = 'path/to/model/file'
model_artifact.properties["version"].int_value = 1
model_artifact.properties["name"].string_value = 'MNIST-v1'
model_artifact.type_id = model_type_id
model_artifact_id = store.put_artifacts([model_artifact])[0]
print(model_artifact_id)

### Record the output event

In [ ]:
# Declare the output event
output_event = metadata_store_pb2.Event()
output_event.artifact_id = model_artifact_id
output_event.execution_id = run_id
output_event.type = metadata_store_pb2.Event.DECLARED_OUTPUT

# Submit output event to the Metadata Store
store.put_events([output_event])

### Mark the execution as completed

In [ ]:
trainer_run.id = run_id
trainer_run.properties["state"].string_value = "COMPLETED"
store.put_executions([trainer_run])


### Group artifacts and executions under a context using attributions and assertions artifacts

In [ ]:
# Create a ContextType, e.g., Experiment with a note property
experiment_type = metadata_store_pb2.ContextType()
experiment_type.name = "Experiment"
experiment_type.properties["note"] = metadata_store_pb2.STRING
experiment_type_id = store.put_context_type(experiment_type)
print(experiment_type_id)

# Group the model and the trainer run to an experiment
my_experiment = metadata_store_pb2.Context()
my_experiment.type_id = experiment_type_id

# Give the experiment a name
my_experiment.name = "exp_" + datetime.utcnow().isoformat("T")
my_experiment.properties["note"].string_value = "My first experiment."
experiment_id = store.put_contexts([my_experiment])[0]
print(my_experiment.name)
print(experiment_id)

attribution = metadata_store_pb2.Attribution()
attribution.artifact_id = model_artifact_id
attribution.context_id = experiment_id

association = metadata_store_pb2.Association()
association.execution_id = run_id
association.context_id = experiment_id

store.put_attributions_and_associations([attribution], [association])

### Check presence of artifacts and executions under the contex

In [ ]:
print("Artifacts for context:\n{}".format(store.get_artifacts_by_context(experiment_id)))
print("Executions for context:\n{}".format(store.get_executions_by_context(experiment_id)))